<a href="https://colab.research.google.com/github/Zontafor/quantum-software/blob/main/L09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Quantum Software Development
# Lab 9: Shor's Factorization Algorithm
# Copyright 2024 The MITRE Corporation. All Rights Reserved.

# Note: Use little endian ordering when storing and retrieving integers from
# qubit registers in this lab.

In [ ]:
!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT, UnitaryGate
from qiskit.visualization import plot_histogram
from math import ceil, log2, pow
from sympy import mod_inverse
import numpy as np

In [ ]:
# E01

def modular_multiply_by_constant(circ, modulus, c, y):
    n = len(y)
    qs = QuantumRegister(n, 'aux')

    # Temporarily add auxiliary register if it doesn't exist
    if 'aux' not in [reg.name for reg in circ.qregs]:
        circ.add_register(qs)

    # Shift and multiply by constant
    for idx in range(n):
        shifted_c = (c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[idx])

    # Swap to target register
    for idx in range(n):
        circ.swap(y[idx], qs[idx])

    # Inverse shift and multiply by inverse constant
    inv_c = mod_inverse(c, modulus)
    for idx in range(n):
        shifted_c = (inv_c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[(idx + shifted_c) % n])

def exp_mod(circ, a, b, input_qubits, output_qubits):
    n = len(input_qubits)

    for i in range(n):
        for j in range(2**i):
            c = (a ** (2**i)) % b
            modular_multiply_by_constant(circ, b, c, output_qubits)

def run_period_finding_circuit(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply Hadamard gates to input qubits
    qc.h(input_qubits)

    # Apply the exp_mod circuit (modular exponentiation)
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Apply the inverse QFT to input qubits
    qc.append(QFT(len(input_qubits), inverse=True).to_instruction(), input_qubits)

    # Measure the input qubits
    qc.measure(input_qubits, classical_bits)

    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    result = sim.run(t_qc, shots=1).result()
    counts = result.get_counts()

    measured_value = max(counts, key=counts.get)
    measurement = int(measured_value.replace(" ", ""), 2)

    return measurement, 2**n

# Example usage
number_to_factor = 15
guess = 7
input_size = ceil(log2(number_to_factor + 1))
output_size = input_size  # Typically, the output register size is similar to the input register size

input_qubits = QuantumRegister(input_size, 'input')
output_qubits = QuantumRegister(output_size, 'output')
classical_bits = ClassicalRegister(input_size + output_size, 'c')

qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

# Apply Hadamard gates to input qubits
qc.h(input_qubits)

# Perform modular exponentiation
exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

# Measure the qubits
qc.measure(input_qubits, classical_bits[:input_size])
qc.measure(output_qubits, classical_bits[input_size:])

# Simulate the circuit
sim = AerSimulator()
t_qc = transpile(qc, sim)
result = sim.run(t_qc).result()
counts = result.get_counts()

print(counts)

In [ ]:
# E02

def modular_exponentiation(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if (exponent % 2) == 1:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

def modular_multiply_by_constant(circ, modulus, c, y):
    n = len(y)
    qs = QuantumRegister(n)
    circ.add_register(qs)

    for idx in range(n):
        shifted_c = (c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[idx])

    for idx in range(n):
        circ.swap(y[idx], qs[idx])

    inv_c = mod_inverse(c, modulus)
    for idx in range(n):
        shifted_c = (inv_c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[(idx + shifted_c) % n])

def exp_mod(circ, a, b, input_qubits, output_qubits):
    classical_bits = ClassicalRegister(len(input_qubits), 'm')
    circ.add_register(classical_bits)

    circ.measure(input_qubits, classical_bits)

    def classical_exp_mod(a, bits, b):
        bitstring = ''.join(map(str, bits[::-1]))
        exponent = int(bitstring, 2)
        return pow(a, exponent, b)

    modular_multiply_by_constant(circ, b, a, output_qubits)
    return classical_exp_mod

def shors_algorithm_subroutine(number_to_factor, guess):
    n = ceil(log2(number_to_factor + 1))
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply Hadamard gates to input qubits
    qc.h(input_qubits)

    # Apply the exp_mod circuit (modular exponentiation)
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Apply the inverse QFT to input qubits
    qc.append(QFT(len(input_qubits), inverse=True).to_instruction(), input_qubits)

    # Measure the input qubits
    qc.measure(input_qubits, classical_bits)

    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    result = sim.run(t_qc, shots=1).result()
    counts = result.get_counts()

    measured_value = max(counts, key=counts.get)
    measurement = int(measured_value.replace(" ", ""), 2)

    return measurement, 2**n

# Example usage
number_to_factor = 15
guess = 7

result = shors_algorithm_subroutine(number_to_factor, guess)
print(f"Result: {result}")

In [ ]:
# Compute fraction, find period, & factor number for E02

from fractions import Fraction

def continued_fraction_approximation(numerator, denominator):
    fraction = Fraction(numerator, denominator)
    return fraction.limit_denominator()

# Example result from the quantum subroutine
numerator = 123
denominator = 16

fraction = continued_fraction_approximation(numerator, denominator)
print(f"Continued fraction approximation: {fraction}")

In [ ]:
# E02


def inverse_qft(qc, qubits):
    qc.append(QFT(len(qubits), inverse=True).to_instruction(), qubits)

def exp_mod(circ, a, b, input_qubits, output_qubits):
    classical_bits = ClassicalRegister(len(input_qubits), 'm')
    circ.add_register(classical_bits)

    circ.measure(input_qubits, classical_bits)

    def classical_exp_mod(a, bits, b):
        bitstring = ''.join(map(str, bits[::-1]))
        exponent = int(bitstring, 2)
        return pow(a, exponent, b)

    circ.append(UnitaryGate(np.eye(2**len(output_qubits))), output_qubits)  # Identity operation as a placeholder

    return classical_exp_mod

def find_approx_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply Hadamard gates to input qubits
    qc.h(input_qubits)

    # Apply the exp_mod circuit (modular exponentiation)
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Apply the inverse QFT to input qubits
    inverse_qft(qc, input_qubits)

    # Measure the input qubits
    qc.measure(input_qubits, classical_bits)

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    result = sim.run(t_qc).result()
    counts = result.get_counts()

    print(counts)
    return counts

# Example usage
number_to_factor = 15
guess = 7
n = ceil(log2(number_to_factor + 1))

find_approx_period(number_to_factor, guess, n)

In [ ]:
# E03

from sympy import Rational
from sympy.ntheory.continued_fraction import continued_fraction_periodic, continued_fraction_convergents

def find_period_candidate(numerator, denominator, denominator_threshold):
    fraction = Rational(numerator, denominator)
    continued_fraction = continued_fraction_periodic(fraction.p, fraction.q)
    convergents = continued_fraction_convergents(continued_fraction)

    candidate = None
    for convergent in convergents:
        if convergent.q < denominator_threshold:
            candidate = convergent
        else:
            break
    return (candidate.p, candidate.q)

# Example usage
numerator = 43
denominator = 256
denominator_threshold = 100

candidate = find_period_candidate(numerator, denominator, denominator_threshold)
print(f"Period candidate: p = {candidate[0]}, q = {candidate[1]}")

In [ ]:
# E04

from sympy import mod_inverse

def modular_multiply_by_constant(circ, modulus, c, y):
    n = len(y)
    qs = QuantumRegister(n)
    circ.add_register(qs)

    # Shift and multiply by constant
    for idx in range(n):
        shifted_c = (c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[idx])

    # Swap to target register
    for idx in range(n):
        circ.swap(y[idx], qs[idx])

    # Inverse shift and multiply by inverse constant
    inv_c = mod_inverse(c, modulus)
    for idx in range(n):
        shifted_c = (inv_c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[(idx + shifted_c) % n])

def exp_mod(circ, a, b, input_qubits, output_qubits):
    classical_bits = ClassicalRegister(len(input_qubits), 'm')
    circ.add_register(classical_bits)

    circ.measure(input_qubits, classical_bits)

    def classical_exp_mod(a, bits, b):
        bitstring = ''.join(map(str, bits[::-1]))
        exponent = int(bitstring, 2)
        return pow(a, exponent, b)

    modular_multiply_by_constant(circ, b, a, output_qubits)
    return classical_exp_mod

def find_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply Hadamard gates to input qubits
    qc.h(input_qubits)

    # Apply the exp_mod circuit (modular exponentiation)
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Measure the output qubits
    qc.measure(output_qubits, classical_bits)

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    result = sim.run(t_qc).result()
    counts = result.get_counts()

    print("Counts:", counts)
    return counts

# Example usage
number_to_factor = 15
guess = 7
n = ceil(log2(number_to_factor + 1))

find_period(number_to_factor, guess, n)

Counts: {'1111 0000': 55, '1001 0000': 64, '0101 0000': 59, '0001 0000': 63, '1110 0000': 54, '0000 0000': 69, '1101 0000': 66, '0110 0000': 66, '0100 0000': 66, '1011 0000': 64, '1000 0000': 68, '0111 0000': 68, '0010 0000': 79, '0011 0000': 65, '1100 0000': 63, '1010 0000': 55}


{'1111 0000': 55,
 '1001 0000': 64,
 '0101 0000': 59,
 '0001 0000': 63,
 '1110 0000': 54,
 '0000 0000': 69,
 '1101 0000': 66,
 '0110 0000': 66,
 '0100 0000': 66,
 '1011 0000': 64,
 '1000 0000': 68,
 '0111 0000': 68,
 '0010 0000': 79,
 '0011 0000': 65,
 '1100 0000': 63,
 '1010 0000': 55}

In [ ]:
# E05

from math import gcd

def modular_exponentiation(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if (exponent % 2) == 1:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

def find_factor(number_to_factor, guess, period):
    if period % 2 != 0:
        # Period is odd, return -1
        return -1

    # Calculate guess^(period/2) % number_to_factor
    x = modular_exponentiation(guess, period // 2, number_to_factor)

    # If x is equivalent to -1 modulo number_to_factor, this period doesn't work for factoring
    if x == number_to_factor - 1:
        return -2

    # Calculate the gcd of (x - 1) and number_to_factor
    factor1 = gcd(x - 1, number_to_factor)
    if factor1 > 1 and factor1 < number_to_factor:
        return factor1

    # Calculate the gcd of (x + 1) and number_to_factor
    factor2 = gcd(x + 1, number_to_factor)
    if factor2 > 1 and factor2 < number_to_factor:
        return factor2

    # If neither factor is found, return -2
    return -2

# Example usage
number_to_factor = 15
guess = 7
period = 4

factor = find_factor(number_to_factor, guess, period)
print(f"Factor found: {factor}")
